<a href="https://colab.research.google.com/github/sylu2305/Domain-Specific-Chatbot/blob/main/Gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Feb 16 17:19:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_id = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-br2e7inzczhe --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true --enable_kernel_event_logging=true ')}
The following directories listed in your path were found to be non-existent: {PosixPath('/datalab/web/pyright/type

/usr/local/lib/python3.11/dist-packages/bitsandbytes/cuda_setup/main.py:167: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/cuda_setup/main.py:167: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
    device = "cuda:0"

    # Ensure the tokenizer is set for left-padding
    tokenizer.padding_side = "left"
    system_prompt = """
    You are a chatbot specifically designed to answer questions related to financial banking topics.
    Your purpose is to provide accurate, concise, and helpful responses to questions about banking services, accounts, loans, transactions, or any financial-related topics.
    If a question is not directly related to financial banking, politely respond with: 'I can only assist with banking-related questions.
    Please ask me something related to banking or finance.' Do not attempt to answer questions outside of this scope.
    """

    # Generate the full response based on banking query
    prompt_template = """
    {system_prompt}

    Query: {query}
    """
    prompt = prompt_template.format(system_prompt=system_prompt,query=query)

    # Encode and generate the final answer from the model
    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return decoded

In [ ]:
from datasets import load_dataset

dataset = load_dataset("bitext/Bitext-retail-banking-llm-chatbot-training-dataset", split="train")
dataset

In [ ]:
df = dataset.to_pandas()
df.head(10)

In [ ]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = """Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"""
    text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model{data_point["response"]} <end_of_turn>"""
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [ ]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
print(test_data)

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
print(model)

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

In [ ]:
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
new_model = "gemma1-Code-Instruct-Finetune-test"

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# # File paths for the three text files
# file1_path = "/content/1.txt"
# file2_path = "/content/2.txt"
# file3_path = "/content/3.txt"

# # Read content from each file and combine
# def extract_personalized_context(file_paths):
#     combined_content = ""
#     for file_path in file_paths:
#         with open(file_path, "r", encoding="utf-8") as file: # Specify UTF-8 encoding
#             content = file.read()
#             combined_content += content + "\n"  # Add a newline between file contents for clarity
#     return combined_content

# # File paths list
# file_paths = [file1_path, file2_path, file3_path]

# # Extract and combine content
# personalized_context = extract_personalized_context(file_paths)

# # Print combined content to verify
# print(personalized_context)

In [ ]:
# from transformers import pipeline

# def summarize_context(context, max_tokens=1024):
#     """
#     Summarizes the context using the 'facebook/bart-large-cnn' model.
#     Truncates the context to a maximum number of tokens to avoid exceeding
#     the model's context window.

#     Args:
#         context (str): The text to summarize.
#         max_tokens (int): The maximum number of tokens to consider.

#     Returns:
#         str: The summary of the context.
#     """
#     summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

#     # Tokenize the context
#     inputs = summarizer.tokenizer(context, return_tensors="pt", truncation=True, max_length=max_tokens)

#     # Generate summary
#     summary_ids = summarizer.model.generate(**inputs, max_length=150, min_length=50, do_sample=False)

#     # Decode summary
#     summary_text = summarizer.tokenizer.decode(summary_ids[0], skip_special_tokens=True)

#     return summary_text

# # Call the modified function
# personalized_context_summary = summarize_context(personalized_context)

In [ ]:
result = get_completion(query="who is ceo of SBI", model=merged_model, tokenizer=tokenizer)
print(result)

In [ ]:
import shutil

# Define source and destination paths
source_folder = '/content/gemma1-Code-Instruct-Finetune-test'
destination_folder = '/content/drive/My Drive/my_colab_folder'
# Move the folder
shutil.move(source_folder, destination_folder)


In [ ]:
import shutil

# Define source and destination paths
source_folder = '/content/merged_model'
destination_folder = '/content/drive/My Drive/merged_model'

# Move the folder
shutil.move(source_folder, destination_folder)

In [ ]:
import shutil

# Define source and destination paths
source_folder = '/content/outputs'
destination_folder = '/content/drive/My Drive/training_outputs'

# Move the folder
shutil.move(source_folder, destination_folder)